# **Fully Connected Neural Network: A `CUDA` and `C++` Implementation**

## **Prepare workspace**

In [15]:
from google.colab import drive
drive.mount("/content/drive")
%cd /content/drive/MyDrive/ParaProgram/Project

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/ParaProgram/Project


In [ ]:
# Extract data from `.gz`
# Only need to run once!
# !pip install patool
# import patoolib
# patoolib.extract_archive("mnist/t10k-images-idx3-ubyte.gz", outdir="mnist")
# patoolib.extract_archive("mnist/t10k-labels-idx1-ubyte.gz", outdir="mnist")
# patoolib.extract_archive("mnist/train-images-idx3-ubyte.gz", outdir="mnist")
# patoolib.extract_archive("mnist/train-labels-idx1-ubyte.gz", outdir="mnist")

## **Edit `Makefile`**

In [3]:
%%writefile Makefile

# Compilers
CXX := g++
CXX_FLAGS := -std=c++17 -ggdb
NVCC := nvcc

# Folders
BIN := bin
SRC := src
INCLUDE := include

EXECUTABLE := nn_main

all: $(BIN)/$(EXECUTABLE)

run: clean all
	clear
	./$(BIN)/$(EXECUTABLE)

$(BIN)/$(EXECUTABLE): $(SRC)/*.cu $(SRC)/*.cpp
	$(NVCC) -I $(INCLUDE) $^ -o $@

clean:
	-rm $(BIN)/*

Overwriting Makefile


## **Compile and run**

In [26]:
# Compile
!make

nvcc -I include src/main.cu src/nn.cu src/utils_device.cu src/utils_host.cpp -o bin/nn_main
src/main.cu(11): warning #177-D: variable "n_hidden" was declared but never referenced
      const int n_hidden = 2;
                ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"



In [27]:
# Run the program
!./bin/nn_main

Train Images: 60000 with size 784 each.
Train Labels: 60000 labels loaded.
Test Images: 10000 with size 784 each.
Test Labels: 10000 labels loaded.
4096
4096
320
Output from the NEW forward pass (first 10 values):
4
1 0 8.72866e-28 0 0 0 0 0 0 0 
Output from the forward pass (first 10 values):
0.00138368 0.000624977 0.98718 0.000225708 1.16301e-05 0.00168083 0.000633362 0.000195483 5.97869e-06 0.00805838 
b3:
